In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from sklearn import preprocessing
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn import metrics

from catboost import CatBoostRegressor
from catboost.utils import eval_metric
from catboost import cv
from catboost import Pool
from sklearn.model_selection import train_test_split
%matplotlib inline
plt.style.use('seaborn')

C:\Users\l-pla\AppData\Local\Temp\ipykernel_3496\606577683.py:18: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


##### Загрузка данных

In [408]:
path = r'data\train_data.zip'
data = pd.read_csv(path, compression = 'zip')

In [409]:
data.head(10)

,product_name,period,price,postcode,address_name,lat,lon,object_type,total_square,rooms,floor,city,settlement,district,area,description,source
0,"3-комнатная, 137 м²",NaN,63000000,127473.0,"2-й Щемиловский переулок, 5а",55.778894,37.608844,Квартира,137.0,3.0,6.0,Москва,NaN,Тверской район,NaN,Просторная квартира свободной планировки с пан...,ЦИАН
1,"Студия, 16,7 м²",NaN,3250000,108815.0,"Харлампиева, 46",55.551025,37.313054,Квартира,16.7,NaN,1.0,Москва,NaN,Филимонковское поселение,NaN,ВНИМАНИЕ! ОЧЕНЬ ПРИВЛЕКАТЕЛЬНОЕ ПРЕ...,Домклик
2,"3-комнатная, 76 м²",NaN,16004680,NaN,"ЖК Прокшино, 8 к4",55.594802,37.431264,Квартира,76.0,3.0,6.0,Москва,NaN,Сосенское поселение,NaN,"Apт.1684018. 0,01% - гибкая ипотека! Воспользу...",Яндекс.Недвижимость
3,"1-комнатная, 24 м²",NaN,7841776,NaN,"ЖК Прокшино, 6 к2",55.594332,37.428099,Квартира,24.0,1.0,10.0,Москва,NaN,Сосенское поселение,NaN,Продается однокомнатная квартира № 381 в новос...,Новострой-М
4,"3-комнатная, 126 м²",NaN,120000000,121352.0,"Давыдковская, 18",55.721097,37.464342,Квартира,126.0,3.0,16.0,Москва,NaN,Фили-Давыдково район,NaN,Шикарное предложение!\nПродаётся трёхкомнатная...,Домклик
5,"3-комнатная, 74 м²",NaN,13300000,142721.0,"Пригородное шоссе, 3",55.557342,37.809447,Квартира,74.0,3.0,14.0,NaN,д. Мисайлово,NaN,NaN,Арт. 32494997 \nОтличная 3-х комнатная квартир...,ЦИАН
6,"2-комнатная, 65 м²",NaN,9500000,143986.0,"Автозаводская, 4 к2",55.740408,38.011392,Квартира,65.0,2.0,3.0,Балашиха,NaN,NaN,Железнодорожный м-н,Предлагается к покупке просторная двухкомнатна...,Домклик
7,"3-комнатная, 87 м²",NaN,15000000,143072.0,"Михаила Кутузова, 15",55.659569,37.218370,Квартира,87.0,3.0,11.0,NaN,пос. ВНИИССОК,NaN,NaN,"ЖК Гусарская Баллада, кирпичный дом, огороженн...",ЦИАН
8,"2-комнатная, 35,8 м²",NaN,7135671,NaN,"3-й квартал, к66",55.700571,37.962692,Квартира,35.8,2.0,11.0,Люберцы,NaN,NaN,Самолёт м-н,Продается двухкомнатная квартира № 542 в новос...,Новострой-М
9,"2-комнатная, 68,3 м²",NaN,31500000,127427.0,"улица Академика Королёва, 21",55.820044,37.602498,Квартира,68.3,2.0,4.0,Москва,NaN,Марфино район,NaN,Дом ''Достижение'' располагается в Москве в ок...,ЦИАН


In [410]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98822 entries, 0 to 98821
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  98822 non-null  object 
 1   period        0 non-null      float64
 2   price         98822 non-null  int64  
 3   postcode      93675 non-null  float64
 4   address_name  98821 non-null  object 
 5   lat           98822 non-null  float64
 6   lon           98822 non-null  float64
 7   object_type   98822 non-null  object 
 8   total_square  98822 non-null  float64
 9   rooms         94840 non-null  float64
 10  floor         98822 non-null  float64
 11  city          91928 non-null  object 
 12  settlement    6894 non-null   object 
 13  district      75111 non-null  object 
 14  area          19498 non-null  object 
 15  description   98573 non-null  object 
 16  source        98822 non-null  object 
dtypes: float64(7), int64(1), object(9)
memory usage: 12.8+ MB


#### EDA

In [411]:
# Отобразим процент пропусков в признаках
data.isna().sum()*100/len(data)

product_name      0.000000
period          100.000000
price             0.000000
postcode          5.208354
address_name      0.001012
lat               0.000000
lon               0.000000
object_type       0.000000
total_square      0.000000
rooms             4.029467
floor             0.000000
city              6.976179
settlement       93.023821
district         23.993645
area             80.269576
description       0.251968
source            0.000000
dtype: float64

In [412]:
# Удалим признак product_name так как вся информация из данного признака содержится в total_square и rooms
data = data.drop('product_name', axis = 1)


##### Заполним пропуски в признаках city и district

In [366]:
# from dadata import Dadata
# token = "c222d559ea7785b09de4e2e39cbe56f2c5e2c3e5"
# dadata = Dadata(token)

# def reverse_geocoding_city (coordinates: pd.Series) -> str:
#     result = dadata.geolocate(name="address", lat=coordinates.iloc[0], lon=coordinates.iloc[1])
#     if len(result)!= 0:
#         return result[0]['data']['city']
#     else: 
#         return np.nan

# def reverse_geocoding_district (coordinates: pd.Series) -> str:
#     result = dadata.geolocate(name="address", lat=coordinates.iloc[0], lon=coordinates.iloc[1])
#     if len(result)!= 0:
#         if result[0]['data']['city_district'] != None:
#             return result[0]['data']['city_district']
#         elif result[0]['data']['settlement'] != None:
#             return result[0]['data']['settlement']
#         else: 
#             return np.nan
#     else: 
#         return np.nan 

In [371]:
# city_none_df = data[data['city'].isna()]
# district_none_df = data[data['district'].isna()]

In [ ]:
# city_none_df  = city_none_df.loc[:,['lat','lon']].apply(reverse_geocoding_city, axis = 1)
# district_none_df  = district_none_df.loc[:,['lat','lon']].apply(reverse_geocoding_district, axis = 1)

In [ ]:
# data[data['city'].isna()] = city_none_df
# data[data['district'].isna()] = district_none_df

###  Законились бесплатные запросы на dadata( Поэтому заполнение пропусков таким образом не получится. 

In [419]:
data[data['city'].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6894 entries, 5 to 98815
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   period        0 non-null      float64
 1   price         6894 non-null   int64  
 2   postcode      6508 non-null   float64
 3   address_name  6894 non-null   object 
 4   lat           6894 non-null   float64
 5   lon           6894 non-null   float64
 6   object_type   6894 non-null   object 
 7   total_square  6894 non-null   float64
 8   rooms         6130 non-null   float64
 9   floor         6894 non-null   float64
 10  city          0 non-null      object 
 11  settlement    6894 non-null   object 
 12  district      1045 non-null   object 
 13  area          2254 non-null   object 
 14  description   6894 non-null   object 
 15  source        6894 non-null   object 
dtypes: float64(7), int64(1), object(8)
memory usage: 915.6+ KB


Заполним city, в которых NaN значениями из признака settlement

In [420]:
data['city'] = data['city'].apply(lambda x: data['settlement'] if data['city'] is np.nan else data['city'])

KeyboardInterrupt: 

In [398]:
data 

,period,price,postcode,address_name,lat,lon,object_type,total_square,rooms,floor,city,settlement,district,area,description,source
0,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва
1,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва
2,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва
3,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва
4,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98817,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва
98818,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва
98819,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва
98820,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва
